# Inference in Gaussian processes

* [mathematicalmonk on GP inference](https://youtu.be/UH1d2mxwet8)

We've seen that a GP is defined by a multivariate normal distribution over *outputs associated with some indices* or *states at some points in time*. Given a number of known states, we can get conditional distributions over one or more unobserved states. We can query the GP for certain indices/times.

Let's separate $Z$ into a known part $Z_b$ and unknown part $Za$:

$$
\begin{pmatrix}
    Z_a \\
    Z_b
\end{pmatrix}
\sim
\mathcal{N}(
    \begin{pmatrix}
        \mu_a \\
        \mu_b
    \end{pmatrix}
,
    \begin{pmatrix}
        K_{aa} && K_{ab} \\
        K_{ba} && K_{bb}
    \end{pmatrix}
)
$$

Given some concrete values $Z_b = z_b$, the conditional is another Gaussian:

$$
(Z_a | Z_b = z_b) \sim \mathcal{N}(m, D)
$$

with

$$
\begin{align}
m &= \mu_a + K_{ab}^T K_{bb}^{-1}(z_b - \mu_b) \\
D &= K_{aa} - K_{ab} K_{bb}^{-1} K_{ba}
\end{align}
$$


## Example: polynomials
Let's transform our linear kernel into a polynomial kernel. If you want to know more about which transforms are allowed for kernels, there's a [video](https://youtu.be/Sc5hOS5HqdY). In this case, we choose a polynomial with non-negative coefficients. Let's look at some samples.

In [ ]:
import numpy as np

from utilities import plot_gp

def plot_3rd_order_poly_gp(coeffs=np.array([0.1, 0.3, 1, 0])):
    t = np.linspace(-5, 5, 10)

    k = lambda i, j: np.polynomial.polynomial.polyval(i * j, coeffs)
    mu = lambda i: 0
    
    plot_gp(mu, k, t, (2, 6))
    
plot_3rd_order_poly_gp()

## Noisy observations

In [ ]:
# TODO

## Analogy to basis functions

In [ ]:
# TODO